<a href="https://colab.research.google.com/github/BecomeAllan/MCTI-scrap/blob/main/MCTI_SCRAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrap functions

Main call para buscar informações dos dados

In [4]:
from Functions.royal import *
from Functions.thegef import *
from Functions.iadb import *
from Functions.ICGEB import *
from Functions.speciesconservation import *
from Functions.ramsar import *
from Functions.globalwomennet import *
from Functions.arcadia import *
from Functions.ruffordj import *

funs = [
        royal1,
        royal2,
        royal3,
        thegef2,
        thegef3,
        iadb4,
        icgeb1,
        icgeb2,
        icgeb3,
        speciesconservation1,
        speciesconservation2,
        speciesconservation3,
        speciesconservation4,
        ramsar2,
        ramsar3,
        globalwomennet1,
        # globalwomennet2,
        globalwomennet3,
        arcadia2,
        arcadia3,
        # arcadia4,
        rufford1,
        rufford2,
        rufford3,
        rufford4
        ]


def call_fun(fun, *args, **keyargs):
  return fun(*args, **keyargs)

# PPFcentral

In [2]:
from datetime import datetime
import filecmp 
from filecmp import dircmp
import os
from os import walk
import pandas as pd
import numpy as np 
import shutil
from itertools import compress
from multiprocessing import Pool, freeze_support
from itertools import repeat
from functools import partial
# Definindo o diretório para salvar os arquivos
# os.chdir(r'D:\Users\beatriz.simoes\Desktop\codigos\mcti-sefip-ppfcd2020\output')
#-------------------------- Criação das pastas -------------------
# PASTA DIARIA
dia = datetime.today().strftime('%y%m%d') # yy/mm/dd
if os.path.exists(dia):
 print('Diretório já existente')   #nada acontece
else:
    os.makedirs(dia) #cria o diretório 
    print('Diretório criado')
# PASTA PRINCIPAL
baseprincipal = './/baseprincipal'
if os.path.exists(baseprincipal):
 print('Diretório já existente')   #nada acontece
else:
    os.makedirs(baseprincipal) #cria o diretório 
    print('Diretório criado')

#---------------------------------------------------------------------------
#-------------------- SCRAPING PARA PASTA DIÁRIA:  -------------------------
#---------------------------------------------------------------------------
#  (aqui ficarão as importações dos arquivos py para scraping diário)

try:
  with Pool(2) as pool:
    pool.map(partial(call_fun, path='.//' + dia), funs)
except:
    print('Erro na extração, verificar arquivo fonte')


#---------------------------------------------------------------------------
#--------------------------------- Funções  -----------------------------
#---------------------------------------------------------------------------
#-----------
# ---- função que define o caminho da pasta------------------
def paths(pasta,arquivo):
    path = '''.\\'''+pasta+'''\\'''+arquivo
    return(path)
#---------------------------------------------------------------------

# --------------------------- função que atualiza a base --------------
def atualizador(baseprincipal,diamaisrecente):
    diario = pd.read_csv(diamaisrecente)
    main = pd.read_csv(baseprincipal)
    # checando o que do 'diario' está no 'main'
    a = diario['link'].isin(main['link'])   # usar o tag/id criado inves do link
    b = [not bool for bool in a] # inverter para o TRUE ser a linha que não tem no main
    novaslinhas = diario[b]
    main = main.append(novaslinhas,ignore_index=True) #novo main
    main.to_csv(baseprincipal,index=False,sep=",")
    print('A base foi atualizada')
# -------------------------------------------------------------------

#---------------------------------------------------------------------------
#--------------------------------- Comparação  -----------------------------
#---------------------------------------------------------------------------

#------------   pegando todos os diretórios da pasta output

_,dirnames,_  = next(walk('.'))
dirnames.sort(reverse=True) # Os obj 0 vai ser a base principal. o 1 vai ser o dia mais recente e o 2 o dia anterior.

_,_,filenamesDia = next(walk('.//'+dia)) # Arquivos extraídos no dia.
_,_,filenamesBase = next(walk('.//baseprincipal')) # Arquivos extraídos na base.

# # -------------------------------- Se o arquivo não tiver na base:
# # Verificando se os nomes que estão no Dia estão na Base.
a = [i in filenamesBase for i in  filenamesDia]
a = [not bool for bool in a] # inversão pro True ser o arquivo faltante
arquivos = list(compress(filenamesDia, a)) # arquivo que está faltando na baseprincipal
for f in arquivos:
    shutil.copy('.//'+dia+'//'+f, './/baseprincipal')
# # ------------------------------------------------------


# #---------------------------------------------------------------------------
# #--------------------------------- Atualização  ----------------------------
# #---------------------------------------------------------------------------
filenames = filenamesDia

for i in range(0,len(filenames)):
    try:
        base = str(paths(dirnames[0],filenames[i])) #base 
        dia1 = paths(dirnames[1],filenames[i]) #dia mais recente 
        dia2 = paths(dirnames[2],filenames[i]) #dia anterior 
        comp = filecmp.cmp(dia1, dia2, shallow = False)
        if comp==False:
            print('arquivos diários são diferentes, base atualizada')
            atualizador(base,dia1)
        else:
            print('arquivos diários são iguais, base não atualizada')
    except:
      print("Arquivo do dia anterior não encontrado")

    print('Concluido arquivo '+str(i+1))

# como a coluna codigo contem o dia, os arquivos sempre vão ser diferentes.


Diretório já existente
Diretório já existente
